# **Seconda challenge del corso di algoritmi per ing. mate**
# Leonardo De Novellis && Francesco Derme

# **Setup**

**Download the code**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 8559, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 8559 (delta 66), reused 108 (delta 43), pack-reused 8405
Receiving objects: 100% (8559/8559), 2.77 MiB | 7.35 MiB/s, done.
Resolving deltas: 100% (5708/5708), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 27461, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27461 (delta 0), reused 4 (delta 0), pack-reused 27455
Receiving objects: 100% (27461/27461), 12.59 MiB | 10.45 MiB/s, done.
Resolving deltas: 100% (20404/20404), done.


**Organize the code and install**

In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Google Benchmark version: v1.8.3-73-gbc946b91, normalized to 1.8.3.73
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST - Success
-- Performing Test HAVE_CXX_FLAG_WCO

# **Challenge 2**

In [ ]:
%%writefile challenge.cpp
#include <cstdio>
#include <vector>
#include <ctime>
#include <benchmark/benchmark.h>

#define fast std::ios_base::sync_with_stdio(0); std::cin.tie(0); std::cout.tie(0);

// IMPORTANTE: il codice seguente assume che fosse possibile fare uso della funzione std::swap
// Se questo non fosse consentito, ci teniamo a far sapere che avremmo saputo implementare uno XOR swap che è
// la migliore opzione in questo caso dopo quella fornita dalla libreria standard.

// Il primo approccio che abbiamo tentato per trovare la mediana dei gruppi da 5 elementi è stato
// un bubblesort parziale del vettore in ingresso che effettuava al massimo 9 swap nel caso pessimo.
// Per ottenere le massime prestazioni teoirche possibili (mediana di 5 elementi in 6 comparazioni, mediana di 4 elementi
// in 5 comparazioni e così via) è stato necessario abbandondare il bubblesort e scrivere funzioni ad hoc per ogni caso.
inline int findMedianOfFive(std::vector<int>& vec, size_t left)
{
	// Inizia il mergesort dei primi 4 elementi ordinando le prime 2 coppie
	if(vec[left] > vec[left + 1]) std::swap(vec[left], vec[left + 1]);
	if(vec[left + 2] > vec[left + 3]) std::swap(vec[left + 2], vec[left + 3]);

	// Confronta i due elementi più piccoli delle prime 2 coppie ed elimina il più piccolo (scambiando anche il partner)
	if(vec[left + 2] < vec[left])
	{
		std::swap(vec[left + 1], vec[left + 3]);
		std::swap(vec[left], vec[left + 2]);
	}

	// Confronta il quinto numero con quello rimasto solo
	if(vec[left + 4] > vec[left + 1]) std::swap(vec[left + 4], vec[left + 1]);

	// Confronta i due elementi più piccoli delle nuove 2 coppie ed elimina il più piccolo  (scambiando anche il partner)
	if(vec[left + 4] < vec[left + 2])
	{
		std::swap(vec[left + 1], vec[left + 3]);
		std::swap(vec[left + 4], vec[left + 2]);
	}

	// Il risultato è l'elemento più piccolo tra quello senza coppia e il più piccolo della coppia rimanente
	return std::min(vec[left + 4], vec[left + 3]);
}

inline int findMedianOfFour(std::vector<int>& vec, size_t left)
{
	if(vec[left] > vec[left + 1]) std::swap(vec[left], vec[left + 1]);
	if(vec[left + 2] > vec[left + 3]) std::swap(vec[left + 2], vec[left + 3]);

	if(vec[left + 2] < vec[left])
	{
		std::swap(vec[left + 1], vec[left + 3]);
		std::swap(vec[left], vec[left + 2]);
	}

	if(vec[left + 3] < vec[left + 1]) std::swap(vec[left + 3], vec[left + 1]);

	return std::min(vec[left + 1], vec[left + 2]);
}

inline int findMedianOfThree(std::vector<int>& vec, size_t left)
{
	if(vec[left] > vec[left + 1]) std::swap(vec[left], vec[left + 1]);
	if(vec[left] > vec[left + 2]) std::swap(vec[left], vec[left + 2]);

	return std::min(vec[left + 1], vec[left + 2]);
}

inline int findMedianOfTwo(std::vector<int>& vec, size_t left)
{
	return std::min(vec[left], vec[left + 1]);
}

int (*findMedian[3])(std::vector<int>& vec, size_t left) = {findMedianOfTwo, findMedianOfThree, findMedianOfFour};

// solve trova ricorsivamente la statistica d'ordine richiesta.
inline int solve(std::vector<int>& vec, size_t left, size_t right, size_t order)
{
	restart:

	// Il caso base è quello in cui è presente un solo elemento, non occorre fare nulla.
	if(left == right - 1) return vec[left];

	// Per trovare un pivot sufficientemente buono in poco tempo si suddivide il vettore
	// in ingresso in gruppi da 5, di ognuno di questi si trova la mediana, il pivot è
	// la mediana tra le mediane dei gruppi, calcolato ricorsivamente.
	size_t groups = (right - left) / 5;
	std::vector<int> median(groups);
	int pivot;

	if(groups)
	{
		for(int i = 0; i < groups; i++)
		{
			median[i] = findMedianOfFive(vec, left + i * 5);
		}

		pivot = solve(median, 0, groups, groups/2);
	}

	// Se non ci sono abbastanza elementi per formare i gruppi, il pivot è semplicemente la mediana.
	else{
		pivot = (*findMedian[right - left - 2])(vec, left);
	}

	// Si ordina il vettore attorno al pivot in modo da determinare l'ordine (rank) del pivot stesso.
	size_t pivotpos = 0;
	for(size_t i=left; i<right; i++){
		if (vec[i] == pivot){
			pivotpos = i;
			break;
		}
	}

	std::swap(vec[left], vec[pivotpos]);

	size_t i = left;
	for(size_t j = left + 1; j < right; j++)
	{
		if(vec[j] <= pivot)
		{
			i++;
			std::swap(vec[i], vec[j]);
		}
	}

	std::swap(vec[left], vec[i]);
	size_t rank = i - left + 1;

	// Se il rank del pivot è maggiore di quello richiesto, occore cercare tra gli elementi più
	// piccoli del pivot. Se al contrario è maggiore, occorre cercare tra gli elementi più grandi
	// del pivot, stando attenti a cambiare l'ordine della statistica ricercata.
	if(rank == order) return pivot;
	else if(rank > order)
	{
		right = i;
		goto restart;
	}
	else
	{
		left = i + 1;
		order = order - rank;
		goto restart;
	}
}

// findOrderStatistic è l'interfaccia della funzione solve, ne rende più comoda
// la chiamata e formatta adeguatamente i risultati.
// void findOrderStatistic(std::vector<int>& vec, size_t order)
// {
// 	size_t dim = vec.size();
// 	if(order < 1 || order > dim) printf("Requested statistic (%zuth largest element) is out of bounds\n", order);
// 	else if(order % 10 == 1 && order != 11) printf("The %zust largest element is %d\n", order, solve(vec, 0, dim, order));
// 	else if(order % 10 == 2) printf("The %zund largest element is %d\n", order, solve(vec, 0, dim, order));
// 	else if(order % 10 == 3) printf("The %zurd largest element is %d\n", order, solve(vec, 0, dim, order));
// 	else printf("The %zuth largest element is %d\n", order, solve(vec, 0, dim, order));

// 	return;
// }

// int main()
// {
// 	fast
// 	std::vector<int> input1 = {2, 3, 4, 2, 6, 4, 3, 2, 56, 7, 1, 1};
// 	std::vector<int> input2 = {1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100};
// 	std::vector<int> input3 = {11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1};
// 	std::vector<int> input4 = {1};
// 	std::vector<int> input5 = {13, 6};
// 	std::vector<int> input6 = {13, 2, 5};
// 	std::vector<int> input7 = {21, 10, 15, 7};
// 	std::vector<int> input8 = {1};
// 	std::vector<int> input9 = {};
// 	std::vector<int> input10 = {-1, 2, -2, -2, 110, -87, 50, 60, 60, 60};
// 	std::vector<int> input11 = {11, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1001,
// 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1001, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1,
// 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1001, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100,
// 100, 100, 100, 1001, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1001, 1, 1, 10, 10, 1,
// 1, 1, 10, 10, 1, 1, 1, 10, 10, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1001, 1, 1, 10, 10, 1, 1, 1, 10, 10, 1, 1, 1, 10, 10, 100};
// 	std::vector<std::vector<int>> inputs = {input1, input2, input3, input4, input5, input6, input7};
//     for(int order = 1; order<1000; order++){
//         findOrderStatistic(input7, order);
//     }
//  	return 0;
//  }


static void BM_order(benchmark::State& state)
{
    int size=state.range(0);
    std::srand(unsigned(std::time(nullptr)));
    std::vector<int> values(size);
    std::generate(values.begin(), values.end(), std::rand);
    size_t x = rand() % values.size() + 1;
    for (auto _ :state)
        benchmark::DoNotOptimize(solve(values, 0, values.size(), x));
    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_order)
    ->RangeMultiplier(2)
    ->Range(1<<2, 1<<20)
    ->Complexity();
BENCHMARK_MAIN();

Overwriting challenge.cpp


In [ ]:
!g++ challenge.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o challenge2

In [ ]:
!./challenge2

2024-04-28T18:45:52+00:00
Running ./challenge2
Run on (2 X 2200.22 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.90, 0.62, 0.50
-----------------------------------------------------------
Benchmark                 Time             CPU   Iterations
-----------------------------------------------------------
BM_order/4             55.9 ns         52.9 ns     13478950
BM_order/8              166 ns          160 ns      4129230
BM_order/16             221 ns          215 ns      2717260
BM_order/32             221 ns          221 ns      3234923
BM_order/64             441 ns          441 ns      1591433
BM_order/128            914 ns          914 ns       768486
BM_order/256           2142 ns         2142 ns       329140
BM_order/512           3978 ns         3978 ns       178933
BM_order/1024          6845 ns         6844 ns       103820
BM_order/2048         14205 ns        14203 ns        

# Testcases e commenti

L'algoritmo è stato testato con vari testcases semplici per verificarne la correttezza, alcuni dei quali sono stati lasciati nel main (commentato, insieme all'interfaccia, per permettere il corretto funzionamento del benchmark). Oltre ad input generali, è stato testato su edge-cases come il vettore nullo, un vettore con un singolo elemento o vettori con numeri negativi.

Grazie al metodo delle mediane, l'algoritmo è lineare (i benchmark variano tra una complessità di ~10N e ~0.6NlgN), e nel caso pessimo una sua singola iterazione riduce l'input di almeno 3*ceil(floor(n/5)/2).

Un testcase del tipo #input2, cioè {1, 1, 1, 100, 100, 1, 1, 1, 100, 100, 1, 1, 1, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100}, con ordine richiesto sufficientemente alto, per esempio k=20, è il caso pessimo per una singola iterazione.

Dai benchmark,